# Post-processing

The historical way to post-process computation is:

1. to know beforehand what will be interesting during the computation
2. generate a specific *POSTPRO.DAT* command file during the pre-processing step
3. during computation, a set of postpro files will be generated in the *POSTPRO* directory
4. then use any means (python recommended) to generate the desired plot/graph/other from these text files

The documentation of the available postpro commands, the needed input and the files generated as well as the different values stored in them is detailed in the **LMGC90_Postpro.pdf** of the **manuals** directory of the LMGC90 distribution.

It is important to note that for some internal value like *SOLVER INFORMATIONS* or *DISSIPATED ENERGY*, there is currently no other way to access these values.

### During pre-processing step
On the pre-processor side, the list of available commands can be retrieve with:

In [ ]:
from pylmgc90 import pre
pre.config.lmgc90dicts.commandList

The management of the postprocessing commands follow the same logic than the other elements of LMGC90's preprocessor:

* create _post commands_ object
* add them to a container
* write the file from the containe

A simple example would be to add a command which will store the contact solver convergence informations at each time step:

In [ ]:
from pathlib import Path

datbox = Path('DATBOX')
datbox.mkdir(exist_ok=True)

# generate an empty container
posts = pre.postpro_commands()

# create the command
sinfo = pre.postpro_command(name='SOLVER INFORMATIONS', step=1)

# add it to container
posts.addCommand(sinfo)

# write the input file for LMGC90
pre.writePostpro(commands=posts, parts=[], path=datbox)

In [ ]:
# command for linux
#!cat DATBOX/POSTPRO.DAT
# command for Windows
#!Get-Content DATBOX\POSTPRO.DAT

A more complete example would be to generate a 2D sample in a box under gravity and wanting to follow the evolution of contact/gravity loads on the lower wall. Let us first generate a simple 2D sample with walls, but stop before writing all the files:

In [ ]:
import math
dim = 2

bodies= pre.avatars()
mods  = pre.models()
mats  = pre.materials()
sees  = pre.see_tables()
tacts = pre.tact_behavs()

mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)
mods.addModel(mod)

tdur = pre.material(name='TDURx',materialType='RIGID',density=1000.)
plex = pre.material(name='PLEXx',materialType='RIGID',density=100.)
mats.addMaterial(tdur,plex)


nb_particles = 1000
radii = pre.granulo_Random(nb_particles, 0.5, 2., seed=0)

# on recupere le plus petit et le plus grand rayon
radius_min=min(radii)
radius_max=max(radii)

lx = 75.
ly = 50. 
nb_remaining_particles, coor, radii = pre.depositInBox2D(radii, lx, ly)

for r, c in zip(radii, coor):
    # creation un nouveau disque rigide, constitue du materiau plex
    body = pre.rigidDisk(r=r, center=c, model=mod, material=plex, color='BLUEx') 
    bodies += body

down = pre.rigidJonc(axe1=0.5*lx+radius_max, axe2=radius_max, center=[0.5*lx, -radius_max],
                     model=mod, material=tdur, color='WALLx')
up   = pre.rigidJonc(axe1=0.5*lx+radius_max, axe2=radius_max, center=[0.5*lx, ly+radius_max],
                     model=mod, material=tdur, color='WALLx')
left = pre.rigidJonc(axe1=0.5*ly+radius_max, axe2=radius_max, center=[-radius_max, 0.5*ly],
                     model=mod, material=tdur, color='WALLx')
right= pre.rigidJonc(axe1=0.5*ly+radius_max, axe2=radius_max, center=[lx+radius_max, 0.5*ly],
                     model=mod, material=tdur, color='WALLx')

left.rotate(psi=-math.pi/2., center=left.nodes[1].coor)
right.rotate(psi=math.pi/2., center=right.nodes[1].coor)

down.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
up.imposeDrivenDof(component=[1, 3], dofty='vlocy')
left.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
right.imposeDrivenDof(component=[2, 3], dofty='vlocy')

bodies += down; bodies += up; bodies += left; bodies += right

ldkdk = pre.tact_behav(name='iqsc0',law='IQS_CLB',fric=0.3)
ldkjc = pre.tact_behav(name='iqsc1',law='IQS_CLB',fric=0.5)
tacts+= ldkjc
tacts+= ldkdk

svdkdk = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=ldkdk,
                       CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='BLUEx',
                       alert=0.1*radius_min)
svdkjc = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=ldkjc,
                       CorpsAntagoniste='RBDY2', antagoniste='JONCx', colorAntagoniste='WALLx',
                       alert=0.1*radius_min)
sees += svdkjc
sees += svdkdk


Then comes the postpro command part. To clarify the next code cells, the container is created anew and the *SOLVER INFORMATIONS* command added again. Then the *TORQUE EVOLUTION* is applied to a list or rigid bodies, thus the creation of the command needs to know on which list of avatars the command must apply:

In [ ]:
posts = pre.postpro_commands()

sinfo = pre.postpro_command(name='SOLVER INFORMATIONS', step=1)
posts.addCommand(sinfo)

torque_evol = pre.postpro_command(name='TORQUE EVOLUTION', rigid_set=[down], step=1)
posts.addCommand(torque_evol)

# file writing:
pre.writeDatbox(dim, mats, mods, bodies, tacts, sees, post=posts, datbox_path=datbox)

### During computation

During the computation, there is only three functions to make sure are called:

* `chipy.OpenPostproFiles` at the end of the initialization step (after all the reading/loading of the database),
* `chipy.WritePostproFiles` at the end of each time step during the computation time loop,
* `chipy.ClosePostproFiles` after the computation time loop.

All these steps are really important, even the last one, which informatically closes all the files which were opened during the initialization step.

In [ ]:
from pylmgc90.chipy import computation

dim = 2

# time evolution parameters
dt = 1e-3
nb_steps = 300

# theta integrator parameter
theta = 0.5

# nlgs parameters
stype = 'Stored_Delassus_Loops       '
norm  = 'Quad ' 
conv  = 1e-4
relax = 1.0
gsit1 = 500
gsit2 = 10

# write parameter
freq_write = 10

# display parameters
freq_display = 10

computation.initialize( dim, dt, theta, h5_file='lmgc90.h5' )
for k in range(1, nb_steps+1):
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, freq_display)
computation.finalize()
print('Finished computation')

At this time there should be _*.DAT_ text files in the *POSTPRO* directory
corresponding to the two commands created.

In [ ]:
# linux or powershell command
#!ls POSTPRO
# windows prompt command
#!dir POSTPRO

### Post-processing

For a simple example let's just trace the y-axis contact force on this wall along time. By simply reading the documentation, one can learn that the first column of the file is the physical time, then reaction along x-axis and finally the reaction along y-axis.


Then to display the curve, the knowledgable user could use a one liner with gnuplot. Otherwise the following with python does the trick:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

fname = 'POSTPRO/TORQUE_0000552.DAT'
torque_value = np.loadtxt(fname)
#torque_value = np.loadtxt(torque_with_e)
plt.plot( torque_value[:,0], torque_value[:,2] )
plt.xlabel('time')
plt.ylabel('Reaction force (y)')
plt.show()